# FM 24 Moneyball

In [24]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import re

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [25]:
### Read raw data

# Select file path for raw data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM 24 exports\Summer 2024\Defensive Midfielders.html"

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM Data Lab outputs\Summer 2024\Defensive Midfielders.csv"

# Ingest in a dataframe 
df_players = pd.read_csv(path)   

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [26]:
df_players.head()

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,entries,leaguemultiplier,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,clear_cut_chances_%,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,key_tackles_%,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,key_headers_%,aerial_challenges_attempted/90,duels_win_%,fouls_committed/90,clearances/90,distance_covered/90,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister
0,27025242,Nicolai Poulsen,"DM, M (C)",30,176 cm,70 kg,-,AGF,3F Superliga,DEN,-,-,Scouting Required,"£4,600 p/w",£300K - £2.9M,-,Right Only,26,2318,6.87,1,85.85,0,1,False,65.58,58.20,89,6.56,10.01,1.24,1.89,1.20,1.84,0.81,0.31,38.10,11.96,3.34,27.92,8.70,9.75,-1.05,-10.77,4.04,3.11,77,0.93,0.08,2.50,2.04,1.40,0.43,0.19,3.49,1.40,40.00,2.10,60.00,0.08,5.56,3.49,59.79,1.98,1.32,13.52,0,14.64,0.99,1.67,11.87,2.87,34,66,56,33,35,21,26,29,30,29,16,35,29,44
1,2000100721,Julius Beck,"DM, M (C)",19,178 cm,62 kg,-,AGF,3F Superliga,DEN,-,Balanced,Media-friendly,"£4,700 p/w",£4M - £8M,-,Right,16,1424,6.78,3,74.95,1,1,False,56.57,50.44,89,5.12,9.05,0.88,1.56,0.88,1.56,0.20,0.19,100.00,11.00,3.48,31.61,8.53,7.46,1.07,14.34,2.91,2.09,72,0.82,0.00,0.00,2.30,2.02,0.44,0.25,2.78,1.14,40.91,1.64,59.09,0.00,0.00,2.78,56.67,1.26,1.71,13.22,0,10.05,0.67,1.20,10.45,2.59,2,44,21,5,7,4,5,5,7,6,3,19,39,11
2,92082660,Michael Akoto,"D (RC), WB (R), DM, M (C)",26,188 cm,72 kg,-,AGF,3F Superliga,GER (GHA),-,-,Scouting Required,"£4,000 p/w",£230K - £2.3M,-,Right,20,1872,6.82,8,66.86,2,1,False,59.47,53.61,90,5.24,8.81,0.58,0.97,0.58,0.97,0.06,0.05,100.00,6.59,2.07,31.39,11.49,6.44,5.05,78.42,2.16,1.68,78,0.48,0.38,22.86,2.65,2.60,0.48,0.24,6.11,3.75,61.42,2.36,38.58,0.14,3.85,6.11,65.70,0.82,0.87,12.26,0,6.88,0.97,0.77,15.45,2.69,18,96,24,19,31,43,22,20,17,32,71,68,96,3
3,53159880,Sivert Mannsverk,"DM, M (C)",22,185 cm,76 kg,Inj,Ajax,Eredivisie,NOR,-,-,Scouting Required,"£12,750 p/w",£12.5M - £20M,-,Right,36,3004,6.82,1,81.19,3,1,False,55.07,47.85,87,5.30,9.63,1.17,2.12,0.96,1.74,1.17,0.36,30.77,10.88,2.97,27.27,9.11,9.68,-0.57,-5.89,3.09,2.04,66,1.05,0.00,0.00,2.24,1.74,0.39,0.21,4.07,2.52,61.76,1.56,38.24,0.39,15.48,4.07,63.60,1.38,0.78,13.01,0,7.49,1.25,1.50,12.86,3.69,18,15,32,19,25,11,37,29,34,23,22,75,52,32
4,37073317,Kenneth Taylor,"DM, M/AM (C)",22,182 cm,66 kg,-,Ajax,Eredivisie,NED,-,Fairly Determined,Level-headed,"£21,500 p/w",£75M - £89M,-,Left,42,3579,7.01,0,85.21,4,1,False,46.47,40.31,87,4.02,8.66,1.28,2.76,0.88,1.89,0.91,0.38,41.67,11.42,3.02,26.43,7.97,8.98,-1.01,-11.25,3.22,2.62,81,0.60,0.03,0.96,5.12,1.56,0.60,0.20,4.85,2.11,43.52,2.74,56.48,0.15,7.14,4.85,58.57,0.63,0.50,13.24,0,10.96,2.44,1.48,11.04,4.95,84,45,35,76,76,81,79,83,83,84,98,56,25,60


In [27]:
# # Keep only natural DMs
# path = 'data/Natural DMs.html'

# # Ingest in a dataframe
# df_dms = pd.read_html(path, encoding='utf-8')[0]

# # Format column names 
# df_dms.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_dms.columns]

# df_dms['natural_dm'] = 'Yes'

# df_dms = df_dms[['uid', 'natural_dm']]

# df_players = df_players.merge(df_dms, 
#                               left_on='id',
#                               right_on='uid',
#                               how='left')

# df_players = df_players[df_players['natural_dm'] == 'Yes']

In [28]:
# Get transfer valuation estimation
def parse_transfer_value(val):
    if pd.isnull(val):
        return None

    # Remove dollar signs and spaces
    val = val.replace('$', '').replace(' ', '')

    # If it's a range like "850K-8.4M"
    if '-' in val:
        low_str, high_str = val.split('-')
        low = parse_single_value(low_str)
        high = parse_single_value(high_str)
        if low is not None and high is not None:
            return (low + high) * (6/10)
    else:
        return parse_single_value(val)

    return None  # fallback

def parse_single_value(s):
    try:
        if s.endswith('M'):
            return float(s[:-1]) * 1_000_000
        elif s.endswith('K'):
            return float(s[:-1]) * 1_000
        else:
            return float(s)  # just in case it's a plain number
    except:
        return None

# Apply to your DataFrame
df_players['price_estimate_M'] = df_players['transfer_value'].apply(parse_transfer_value) / 1000000

## Plot function

In [29]:
def scatter_plot(df, x_metric, y_metric, z_metric, 
                 title, show_color_bar=False, show_name_label=True): 
    # Set text to 'name' if show_name_label is True, else empty string
    text_col = 'name' if show_name_label else None

    # Create plotly figure
    fig = px.scatter(
        df,
        x=x_metric,
        y=y_metric,
        color=z_metric,
        text=text_col,
        color_continuous_scale='RdYlGn',
        labels={z_metric: f'Color Scale ({z_metric})'},
        title=title,
        width=1200, 
        height=700    
    )

    # Format traces
    fig.update_traces(
        marker=dict(size=10), 
        textposition='top center',
        textfont=dict(size=10),
        customdata=df[['name', 'club']],  # Always pass 'name' for hovertemplate
        hovertemplate=(
            "<b>%{customdata[0]}</b><br>" +
            "<b>%{customdata[1]}</b><br><br>" +
            f"{x_metric}: " + "%{x}<br>" +
            f"{y_metric}: " + "%{y}<br>" +
            f"{z_metric}: " + "%{marker.color}<extra></extra>"
        )
    )

    fig.update_layout(
        title={
            'text': f'{title} ({len(df): ,.0f})',
            'x': 0.5,
            'xanchor': 'center'
        },
        coloraxis_colorbar=dict(title=z_metric),
        coloraxis_showscale=show_color_bar,  # Show or hide the color bar
        plot_bgcolor='darkgray',
        paper_bgcolor='darkgray',
        xaxis=dict(showgrid=True, gridcolor='white', zeroline=False),
        yaxis=dict(showgrid=True, gridcolor='white', zeroline=False)
    )

    fig.show()


## Passing analysis

### Full universe

In [30]:
# Initialize list of targets
targets = df_players['id']

scatter_plot(df_players,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=False
)

In [31]:
# Filter data 
mask = (
        (df_players['passes_attempted/90'] >= 45) & \
        (df_players['progressive_pass_rate'] >= 7) & \
        (df_players['pass_completion_%'] >= 85) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Possession Analysis

In [32]:
# Full universe 
scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=False
)

In [33]:
# Filter data 
mask = (
        (df_players['possession_lost/90'] <= 8) & \
        (df_players['possession_won/90'] >= 7.5) & \
        (df_players['poss+_/90'] >= 0) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Aggresion vs Control
Identify aggresive and foul prones targets

In [34]:
# Full universe 
scatter_plot(df_targets,
             x_metric='tackles_completed/90', 
             y_metric='fouls_committed/90',
             z_metric='tackle_completion_%',
             title='Aggresion - Control', 
             show_name_label=False
)

In [35]:
df_players[df_players['name'] == 'Casemiro']

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,entries,leaguemultiplier,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,clear_cut_chances_%,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,key_tackles_%,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,key_headers_%,aerial_challenges_attempted/90,duels_win_%,fouls_committed/90,clearances/90,distance_covered/90,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M
287,19061518,Casemiro,"DM, M (C)",32,185 cm,84 kg,-,Man Utd,Premier League,BRA (ESP),-,Spirited,Evasive,"£300,000 p/w",£41M - £51M,-,Right,35,2888,6.78,11,62.78,287,1,False,59.15,53.38,90,4.49,7.59,0.84,1.42,0.84,1.42,0.31,0.16,50.0,8.94,2.4,26.83,8.48,6.95,1.53,22.01,3.49,2.59,74,0.9,0.12,4.82,1.81,1.46,0.37,0.22,3.8,2.59,68.03,1.22,31.97,0.62,24.1,3.8,70.94,1.93,0.87,13.55,1,13.65,0.78,1.06,15.84,2.31,10,67,13,5,7,3,7,5,8,4,0,83,72,18,NaN


In [36]:
# Compute tackes/fouls ratio
df_players['tackles_comp-fouls/90'] = df_players['tackles_completed/90'] / df_players['fouls_committed/90']
# Filter data 
mask = (
        (df_players['tackles_comp-fouls/90'] <= 2) & \
        (df_players['tackles_completed/90'] >= 1.5) & \
        (df_players['tackle_completion_%'] >= 65) & \
        #(df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='tackles_completed/90', 
             y_metric='fouls_committed/90',
             z_metric='tackle_completion_%',
             title='Aggresion - Control', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Pressure quality

In [37]:
# Full universe 
scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

In [152]:
# Filter data 
mask = (
        (df_players['pressures_completed/90'] >= 2.5) & \
        (df_players['pressure_success_%'] >= 20) & \
        (df_players['duels_win_%'] >= 50) & \
        (df_players['id'].isin(targets)) | \
        (df_players['name'].str.contains('Barreiro')) 
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Presence

In [153]:
df_players.head(1)

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,distance_covered/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,uid,natural_dm,price_estimate_M,tackles_comp-fouls/90
1,211515,Baptiste Santamaria,"DM, M/AM (C)",28,183 cm,81 kg,-,Rennes,Ligue 1 Uber Eats,FRA,-,Resolute,Level-headed,"$16,000 p/w",$10M - $12.5M,-,Right,20,1741,6.87,1,82.9,1,73.56,66.07,90,8.27,11.24,1.03,1.41,0.98,1.34,0.07,0.05,12.25,3.31,27.0,8.22,9.15,-0.93,-10.16,4.03,2.95,73,1.09,0.0,2.36,1.71,0.47,0.26,3.67,1.09,29.58,2.58,70.42,0.1,3.67,52.35,0.47,1.71,0.72,13.79,1.12,1.19,11.02,3.36,47,100,38,63,47,28,18,40,29,32,19,6,31,26,20,211515.0,Yes,13.5,1.725146


In [154]:
# Full universe 
scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
) 

In [155]:
# Filter data 
mask = (
        (df_players['interceptions/90'] >= 1.5) & \
        (df_players['blocks/90'] >= 0.2) & \
        (df_players['clearances/90'] >= 0.5) & \
        (df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
)

#df_targets = df_players[mask]

## Distance

In [156]:
df_players.head(1)

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,distance_covered/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,uid,natural_dm,price_estimate_M,tackles_comp-fouls/90
1,211515,Baptiste Santamaria,"DM, M/AM (C)",28,183 cm,81 kg,-,Rennes,Ligue 1 Uber Eats,FRA,-,Resolute,Level-headed,"$16,000 p/w",$10M - $12.5M,-,Right,20,1741,6.87,1,82.9,1,73.56,66.07,90,8.27,11.24,1.03,1.41,0.98,1.34,0.07,0.05,12.25,3.31,27.0,8.22,9.15,-0.93,-10.16,4.03,2.95,73,1.09,0.0,2.36,1.71,0.47,0.26,3.67,1.09,29.58,2.58,70.42,0.1,3.67,52.35,0.47,1.71,0.72,13.79,1.12,1.19,11.02,3.36,47,100,38,63,47,28,18,40,29,32,19,6,31,26,20,211515.0,Yes,13.5,1.725146


In [157]:
# Full universe 
scatter_plot(df_targets,
             x_metric='distance_covered/90', 
             y_metric='defensive_actions/90',
             z_metric='creative_actions/90',
             title='Distance', 
             show_name_label=True
) 

In [160]:
df_targets.groupby('division')['id'].count()

division
3F Superliga                            1
Bundesliga                              3
Campeonato Brasileiro Série A Assaí    14
Eredivisie                              2
LaLiga EA Sports                        3
Liga Portugal Betclic                   3
Liga Profesional de Fútbol             14
Ligue 1 Uber Eats                       2
Premier League                          1
Serie A TIM                             2
Sky Bet Championship                    3
Name: id, dtype: int64

In [170]:
df_targets.groupby('division').get_group('Sky Bet Championship').sort_values('age')

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,distance_covered/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,uid,natural_dm,price_estimate_M,tackles_comp-fouls/90
258,29156436,Flynn Downes,"DM, M (C)",24,182 cm,79 kg,-,Southampton,Sky Bet Championship,ENG,Trained in nation (15-21),-,Scouting Required,"$37,500 p/w",$9.4M - $15M,-,Right Only,24,2080,6.93,0,86.67,258,62.18,56.25,90,7.70,12.39,0.82,1.32,0.82,1.32,0.27,0.26,8.39,2.64,31.44,8.70,6.97,1.73,24.82,3.63,2.86,79,0.78,0.04,1.83,1.43,0.26,0.13,2.51,1.43,56.90,1.08,43.10,0.22,2.51,69.72,0.82,1.99,0.61,13.33,1.35,1.30,11.01,2.42,78,100,28,65,69,40,58,41,52,33,49,17,39,23,81,29156436.0,Yes,14.640,1.437186
263,29170352,David Kasumu,"D/WB (R), DM, M (C)",24,174 cm,60 kg,-,Huddersfield,Sky Bet Championship,NGA (ENG),Trained in nation (15-21),-,Scouting Required,"$7,500 p/w",$300K - $3.1M,-,Right,21,1802,6.79,0,85.81,263,45.90,40.31,88,4.30,9.36,0.95,2.07,0.95,2.07,0.26,0.25,10.19,2.75,26.96,9.94,7.44,2.50,33.60,3.70,2.60,70,1.10,0.05,1.85,1.65,0.70,0.55,3.30,0.95,28.79,2.35,71.21,0.25,3.30,50.71,0.50,2.00,1.25,13.35,1.24,1.30,12.50,2.85,43,100,42,21,29,22,14,23,28,43,22,41,40,51,59,29170352.0,Yes,2.040,1.300000
435,67228750,Keidi Bare,"DM, M (C)",26,174 cm,66 kg,Wnt,Coventry,Sky Bet Championship,ALB,-,-,Scouting Required,"$8,000 p/w",$190K - $1.9M,-,Right,18,1320,6.88,0,73.33,435,51.61,46.43,90,5.45,10.57,1.16,2.25,1.02,1.98,0.20,0.20,11.80,3.07,26.01,10.02,6.95,3.07,44.17,4.23,3.20,76,1.02,0.00,2.07,2.66,0.75,0.34,3.14,1.16,36.96,1.98,63.04,0.14,3.14,59.26,0.41,2.05,1.30,13.43,1.85,1.36,13.23,4.50,54,100,87,73,57,54,48,50,44,40,47,66,32,79,11,67228750.0,Yes,1.254,1.560976
